In [14]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/hw2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/hw2'
/content/drive/My Drive/hw2


In [15]:
!ls

d2v_default.p	      download_ap.ipynb    __pycache__	     trec.py
datasets	      environment.yml	   read_ap.py	     WINDOWS.md
Doc2vec_default.json  hw2_description.pdf  requirements.txt
Dov2Vec2.ipynb	      processed_docs.pkl   tf_idf.py


In [0]:
# ! pip install pytrec-eval

In [23]:
import os
import gensim
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
# import trec
import pytrec_eval
import json


import read_ap
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
def rank(query, model):
      inferred_vector = model.infer_vector(query, 10000)
      sims = model.docvecs.most_similar([inferred_vector], topn = len(model.docvecs))
      return sims
def print_similar(query, model, traincorpus):
  query = read_ap.process_text(query)
  sims = rank(query, model)
  # print('Document ({}): «{}»\n'.format(doc_id, ' '.join(traincorpus[doc_id].words)))
  print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
  for label, index in [('1', 0), ('2', 1), ('3', 2), ('4', 3), ('5', 4), ('6', 5), ('7', 6), ('8', 7), ('9', 8), ('10', 9)]:
      print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(traincorpus[sims[index][0]])))

def main():

    qrels, queries = read_ap.read_qrels()
    docs_by_id = read_ap.get_processed_docs()

    traincorpus = [TaggedDocument(j, [i]) for i, j in docs_by_id.items()]

    model = gensim.models.doc2vec.Doc2Vec(min_count=50, epochs=5)
    model.build_vocab(traincorpus)
    model.train(traincorpus, total_examples=model.corpus_count, epochs=model.epochs)
    # vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
    # print(vector)

    overall_ser = {}

    print("Running Doc2Vec Benchmark")
    # collect results
    for qid in tqdm(qrels): 
        query_text = queries[qid]
        query_text = read_ap.process_text(query_text)
        # print(query_text)
        sims = rank(query_text, model)
        overall_ser[qid] = dict(sims)
    print(list(overall_ser.keys())[0])
    
    return qrels, overall_ser, model, traincorpus

if __name__ == "__main__":
    qrels, overall_ser, model, traincorpus = main()
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'ndcg'})
    metrics = evaluator.evaluate(overall_ser)
    print(evaluator)
    MAP = {k: v['map'] for k, v in metrics.items()}
    all_mean_map = np.array([MAP[k] for k in MAP]).mean()
    NDCG = {k: v['ndcg'] for k, v in metrics.items()}
    all_mean_ndcg = np.array([NDCG[k] for k in NDCG]).mean()


    print(all_mean_map)
    print(all_mean_ndcg)

    print(metrics)

    # dump this to JSON
    # *Not* Optional - This is submitted in the assignment!
    with open("Doc2vec_default.json", "w") as writer:
        json.dump(metrics, writer, indent=1)
    pickle.dump(model, open( "d2v_default.p", "wb"))
    model = pickle.load(open( "d2v_default.p", "rb" ))
    query = 'Donald John Trump (born June 14, 1946) is the 45th and current president of the United States. Before entering politics, he was a businessman and television personality. Trump was born and raised in Queens, a borough of New York City, and received a bachelors degree in economics from the Wharton School. He took charge of his familys real-estate business in 1971, renamed it The Trump Organization, and expanded its operations from Queens and Brooklyn into Manhattan. The company built or renovated skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name.'
    print_similar(query, model, traincorpus)
    MAP_val = {k: v['map'] for k, v in metrics.items() if 67<k<100}
    val_mean = np.array([MAP_val[k] for k in MAP_val]).mean()
    NDCG_val = {k: v['ndcg'] for k, v in metrics.items() if 67<k<100}
    val_mean_NDCG = np.array([NDCG_val[k] for k in NDCG_val]).mean()

    print(val_mean)
    print(val_mean_NDCG)





Docs already processed. Loading from disk


  0%|          | 0/149 [00:00<?, ?it/s]

Running Doc2Vec Benchmark


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 149/149 [01:18<00:00,  1.90it/s]


101
0.0032917703597724725
0.2745544842853375
{'101': {'map': 0.0006571780638420233, 'ndcg': 0.21339336844130485}, '102': {'map': 7.747836899550373e-05, 'ndcg': 0.16673385384700332}, '103': {'map': 0.0003378642392231819, 'ndcg': 0.2513800593730878}, '104': {'map': 0.0002934804070127914, 'ndcg': 0.2429406248008837}, '105': {'map': 0.00011160512496370194, 'ndcg': 0.19780632200800008}, '106': {'map': 0.000195417452691984, 'ndcg': 0.22760499128135164}, '107': {'map': 0.005165598678784621, 'ndcg': 0.2943136338402304}, '108': {'map': 0.001602701978848705, 'ndcg': 0.3270654571233905}, '109': {'map': 0.000114785467360329, 'ndcg': 0.13341761535388602}, '110': {'map': 0.005441422740984226, 'ndcg': 0.4600846118854865}, '111': {'map': 0.0005062038627885934, 'ndcg': 0.30089899597492403}, '112': {'map': 0.00015372388936563225, 'ndcg': 0.1572919468988453}, '113': {'map': 0.00016140947807480983, 'ndcg': 0.20559178649368795}, '114': {'map': 0.0005692326962657107, 'ndcg': 0.3115980010578866}, '115': {'ma

TypeError: ignored